In [8]:
import cv2
import numpy as np

# 加载图像
image_path = './111.png'
image = cv2.imread(image_path)

# 定义颜色范围 (使用 HSV 格式)
# 根据扫雷游戏常见颜色来设置范围，这些范围可以根据实际图像进一步调整
color_ranges = {
    '1': ((102, 159, 199), (122, 199, 255)),  # 将范围设为中心值的 ±10、±20 和 ±30
    '2': ((82, 165, 82), (102, 185, 102)),  # 绿色范围
}

# 初始化矩阵
num_rows, num_cols = 16, 30
matrix = [['H' for _ in range(num_cols)] for _ in range(num_rows)]

# 将图像转换为 HSV 颜色空间
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 定义网格单元的大小
cell_height = hsv_image.shape[0] // num_rows
cell_width = hsv_image.shape[1] // num_cols

# 遍历每个网格单元并进行颜色检测
for row in range(num_rows):
    for col in range(num_cols):
        # 计算每个单元格的坐标
        x_start = col * cell_width
        y_start = row * cell_height
        x_end = x_start + cell_width
        y_end = y_start + cell_height

        # 截取每个单元格区域
        cell = hsv_image[y_start:y_end, x_start:x_end]

        # 初始化标记为隐藏 ('H')
        detected_value = 'H'
        
        # 检测每种颜色范围，并确定数字
        for num, (lower, upper) in color_ranges.items():
            # 创建掩膜来检测颜色范围内的像素
            mask = cv2.inRange(cell, np.array(lower), np.array(upper))
            # 如果检测到足够多的像素，则认为该单元格包含该数字
            if cv2.countNonZero(mask) > (cell_height * cell_width * 0.1):  # 20% 以上像素符合颜色范围
                detected_value = num
                break
        
        # 将检测到的值填入矩阵
        matrix[row][col] = detected_value


In [9]:
# 打印矩阵，每行作为一行输出
for row in matrix:
    print(' '.join(row))


H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H


In [10]:
# 遍历每个网格单元并进行颜色检测
for row in range(num_rows):
    for col in range(num_cols):
        # 计算每个单元格的坐标
        x_start = col * cell_width
        y_start = row * cell_height
        x_end = x_start + cell_width
        y_end = y_start + cell_height

        # 截取每个单元格区域
        cell = hsv_image[y_start:y_end, x_start:x_end]

        # 计算平均颜色值
        avg_color = cv2.mean(cell)[:3]  # 提取平均颜色的 HSV 值

        # 初始化标记为隐藏 ('H')
        detected_value = 'H'
        
        # 检测每种颜色范围，并确定数字
        for num, (lower, upper) in color_ranges.items():
            # 创建掩膜来检测颜色范围内的像素
            mask = cv2.inRange(cell, np.array(lower), np.array(upper))
            # 如果检测到足够多的像素，则认为该单元格包含该数字
            if cv2.countNonZero(mask) > (cell_height * cell_width * 0.1):  # 20% 以上像素符合颜色范围
                detected_value = num
                break
        
        # 将检测到的值和平均颜色填入矩阵
        matrix[row][col] = (detected_value, avg_color)  # 存储数字和颜色值

# 打印矩阵及其颜色值
for row in matrix:
    print(row)


[('H', (104.0, 59.799586776859506, 64.2004132231405)), ('H', (103.96487603305785, 74.12396694214877, 89.34504132231405)), ('H', (103.85950413223141, 74.26652892561984, 90.6900826446281)), ('H', (89.73347107438016, 77.65082644628099, 92.06198347107438)), ('H', (103.85950413223141, 74.26652892561984, 90.6900826446281)), ('H', (103.96487603305785, 74.12396694214877, 89.34504132231405)), ('H', (104.04545454545455, 61.053719008264466, 63.12809917355372)), ('H', (104.13223140495867, 61.010330578512395, 63.171487603305785)), ('H', (103.85950413223141, 74.26652892561984, 90.6900826446281)), ('H', (104.25826446280992, 48.79338842975207, 81.43181818181819)), ('H', (104.40289256198348, 50.28512396694215, 80.47520661157024)), ('H', (104.40289256198348, 50.29338842975207, 80.08677685950414)), ('H', (104.40289256198348, 50.28512396694215, 80.47520661157024)), ('H', (104.40289256198348, 50.29338842975207, 80.08677685950414)), ('H', (93.01859504132231, 74.92975206611571, 86.54958677685951)), ('H', (96

In [31]:
import cv2
import numpy as np

# 加载图像
image_path = './111.png'
image = cv2.imread(image_path)

# 定义颜色范围 (使用 HSV 格式)
color_ranges = {
    '0': ((100, 53, 60), (110, 67, 70)), # 扩大灰色范围
    '1': ((100, 69, 85), (110, 79, 95)),  # 扩大颜色范围
    '2': ((80, 160, 80), (105, 190, 105)),   # 扩大绿色范围
}

# 初始化矩阵
num_rows, num_cols = 16, 30
matrix = [['H' for _ in range(num_cols)] for _ in range(num_rows)]

# 将图像转换为 HSV 颜色空间
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 定义网格单元的大小
cell_height = hsv_image.shape[0] // num_rows
cell_width = hsv_image.shape[1] // num_cols

# 遍历每个网格单元并进行颜色检测
for row in range(num_rows):
    for col in range(num_cols):
        # 计算每个单元格的坐标
        x_start = col * cell_width
        y_start = row * cell_height
        x_end = x_start + cell_width
        y_end = y_start + cell_height

        # 截取每个单元格区域
        cell = hsv_image[y_start:y_end, x_start:x_end]

        # 计算平均颜色值
        avg_color = cv2.mean(cell)[:3]  # 提取平均颜色的 HSV 值

        # 初始化标记为隐藏 ('H')
        detected_value = 'H'

        # 根据平均颜色值决定矩阵的值
        for num, (lower, upper) in color_ranges.items():
            h_check = lower[0] <= avg_color[0] <= upper[0]
            s_check = lower[1] <= avg_color[1] <= upper[1]
            v_check = lower[2] <= avg_color[2] <= upper[2]
            
            #print(f"Checking {num}: H: {h_check}, S: {s_check}, V: {v_check}")

            if h_check and s_check and v_check:
                detected_value = num
                break

        # 将检测到的值和平均颜色填入矩阵
        matrix[row][col] = (detected_value, avg_color)  # 存储数字和颜色值

# 打印矩阵
for row in matrix:
    print(row)


[('0', (104.0, 59.799586776859506, 64.2004132231405)), ('1', (103.96487603305785, 74.12396694214877, 89.34504132231405)), ('1', (103.85950413223141, 74.26652892561984, 90.6900826446281)), ('H', (89.73347107438016, 77.65082644628099, 92.06198347107438)), ('1', (103.85950413223141, 74.26652892561984, 90.6900826446281)), ('1', (103.96487603305785, 74.12396694214877, 89.34504132231405)), ('0', (104.04545454545455, 61.053719008264466, 63.12809917355372)), ('0', (104.13223140495867, 61.010330578512395, 63.171487603305785)), ('1', (103.85950413223141, 74.26652892561984, 90.6900826446281)), ('H', (104.25826446280992, 48.79338842975207, 81.43181818181819)), ('H', (104.40289256198348, 50.28512396694215, 80.47520661157024)), ('H', (104.40289256198348, 50.29338842975207, 80.08677685950414)), ('H', (104.40289256198348, 50.28512396694215, 80.47520661157024)), ('H', (104.40289256198348, 50.29338842975207, 80.08677685950414)), ('H', (93.01859504132231, 74.92975206611571, 86.54958677685951)), ('H', (96